In [1]:
import os
import torch
import random
import numpy as np
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from io import BytesIO
from base64 import b64decode
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import albumentations as A
from albumentations.pytorch import ToTensorV2
from xgboost import XGBRegressor

In [2]:
df = pd.read_csv("../../post2ctr_dataset.csv")

max_size = 224

transform = A.Compose([
    A.Resize(max_size, max_size),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
        
    A.RandomBrightnessContrast(p=0.75),
    A.ShiftScaleRotate(rotate_limit=360,shift_limit=0.15,scale_limit=0.1,p=0.75),
    A.OneOf([
        A.GaussNoise(var_limit=[10, 50]),
        A.GaussianBlur(),
        A.MotionBlur(),
    ], p=0.4),
    A.CoarseDropout(max_holes=2, max_width=int(max_size * 0.2), max_height=int(max_size * 0.2), mask_fill_value=0, p=0.5),
    
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(transpose_mask=True),
])

class TemplateDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        cell = self.df.iloc[idx]
        
        img = Image.open(BytesIO(b64decode(cell["photo"])))
        img = np.asarray(img.convert("RGB"))
        img = self.transform(image=img)["image"]
        
        target = cell["open_photo"] / cell["view"] if cell["view"] > 0 else 0

        return {
            "img" : img,
            "target" : target
        }

In [3]:
seed = 42
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
train_df, test_df = train_test_split(df, test_size=0.9, random_state=42)

train_dataset = TemplateDataset(train_df, transform=transform)
test_dataset = TemplateDataset(test_df, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

In [4]:
class Extractor(nn.Module):
    def __init__(self):
        super(Extractor, self).__init__()
        image_model = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_efficientnet_b0', pretrained=True)
        self.feature_extractor = nn.Sequential(
            *list(image_model.children())[:-1]
        )
        self.flatten = nn.Flatten()

    def forward(self, image_input):
        features = self.feature_extractor(image_input)
        features = self.flatten(features)
        return features

In [5]:
model = Extractor()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Using cache found in /home/vivy/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub
/home/vivy/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub/PyTorch/Classification/ConvNets/image_classification/models/common.py:13: UserWarning: pytorch_quantization module not found, quantization will not be available
  warnings.warn(
/home/vivy/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub/PyTorch/Classification/ConvNets/image_classification/models/efficientnet.py:17: UserWarning: pytorch_quantization module not found, quantization will not be available
  warnings.warn(


Extractor(
  (feature_extractor): Sequential(
    (0): Sequential(
      (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (activation): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): Sequential(
        (block0): MBConvBlock(
          (depsep): Sequential(
            (conv): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (bn): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
            (act): SiLU(inplace=True)
          )
          (se): SequentialSqueezeAndExcitation(
            (squeeze): Linear(in_features=32, out_features=8, bias=True)
            (expand): Linear(in_features=8, out_features=32, bias=True)
            (activation): SiLU(inplace=True)
            (sigmoid): Sigmoid()
            (mul_a_quantizer): Ident

In [6]:
def get_features(model, loader):
    model.eval()
    features = []
    targets = []
    with torch.no_grad():
        for batch_idx, data in enumerate(loader, 0):
            images = data["img"].to(device)
            target = data["target"].cpu().numpy()
                        
            outputs = model(images).cpu().numpy()
            features.append(outputs)
            targets.append(target)

    features = np.concatenate(features, axis=0)
    targets = np.concatenate(targets, axis=0)

    return features, targets

train_features, train_targets = get_features(model, train_loader)

# hist, bin_edges = np.histogram(train_targets, bins=50, range=(0, 1))
# bin_edges = bin_edges[1:]
# bin_assignments = np.digitize(train_targets, bin_edges, right=True)
# train_weights = np.zeros_like(train_targets)

# for i, bin_id in enumerate(bin_assignments):
#     train_weights[i] = 1 / hist[bin_id - 1] if hist[bin_id - 1] > 0 else 1.0
# train_weights /= np.mean(train_weights)

test_features, test_targets = get_features(model, test_loader)

In [7]:
xgb_model = XGBRegressor(n_estimators=1000, max_depth=7, eta=0.1, subsample=0.7, colsample_bytree=0.8, objective='reg:squarederror')
xgb_model.fit(train_features, train_targets, verbose=100)

: 

In [ ]:
preds = xgb_model.predict(test_features)
loss = np.sqrt(((preds - test_targets) ** 2).mean())
print(f"RMSE on test : {loss}")

In [ ]:
xgb_model.save_model("../../effnet_xgboost.json")